# Lab: Comparing Regression Models


## Week 4 Lab Work

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
marketing_df = pd.read_csv('marketing_customer_analysis_clean.csv')
marketing_df = marketing_df.drop('unnamed:_0',axis=1)
marketing_df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


In [3]:
marketing_df.rename(columns=lambda name:name.lower().replace(' ','_'), inplace=True)
marketing_df.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size', 'vehicle_type', 'month'],
      dtype='object')

In [4]:
# divide df into numerical and categorical dfs
marketing_numerical = marketing_df.select_dtypes(include='number')
marketing_categorical = marketing_df.select_dtypes(include='object')

In [5]:
# apply one hot encoding to nominal values
marketing_nominal = pd.get_dummies(marketing_categorical[['state','employmentstatus','gender','location_code',
                                                          'marital_status','policy_type','policy','renew_offer_type',
                                                          'sales_channel','vehicle_class']],
                                   dtype=int)

In [6]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
ord_enc = OrdinalEncoder()
oh_enc = OneHotEncoder()

In [7]:
marketing_ordinal = marketing_df[['response', 'coverage', 'education', 'vehicle_size']]

In [8]:
# apply OrdinalEncoder to ordinal df
response_enc = OrdinalEncoder(categories=[['No','Yes']])
marketing_ordinal['response'] = response_enc.fit_transform(marketing_ordinal[['response']])

coverage_enc = OrdinalEncoder(categories=[['Basic','Extended','Premium']])
marketing_ordinal['coverage'] = coverage_enc.fit_transform(marketing_ordinal[["coverage"]])

education_enc = OrdinalEncoder(categories=[['High School or Below','College','Bachelor','Master','Doctor']])
marketing_ordinal['education'] = education_enc.fit_transform(marketing_ordinal[['education']])

vehicle_enc = OrdinalEncoder(categories=[['Small','Medsize','Large']])
marketing_ordinal['vehicle_size'] = vehicle_enc.fit_transform(marketing_ordinal[['vehicle_size']])

marketing_ordinal

/var/folders/w8/yzcfltnn5pg6fwc5ty1v2rnc0000gn/T/ipykernel_17943/1035201458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketing_ordinal['response'] = response_enc.fit_transform(marketing_ordinal[['response']])
/var/folders/w8/yzcfltnn5pg6fwc5ty1v2rnc0000gn/T/ipykernel_17943/1035201458.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketing_ordinal['coverage'] = coverage_enc.fit_transform(marketing_ordinal[["coverage"]])
/var/folders/w8/yzcfltnn5pg6fwc5ty1v2rnc0000gn/T/ipykernel_17943/1035201

,response,coverage,education,vehicle_size
0,0.0,0.0,1.0,1.0
1,0.0,0.0,1.0,1.0
2,0.0,0.0,2.0,1.0
3,1.0,1.0,1.0,1.0
4,0.0,2.0,2.0,1.0
...,...,...,...,...
10905,0.0,2.0,2.0,1.0
10906,0.0,0.0,1.0,1.0
10907,0.0,1.0,2.0,1.0
10908,0.0,2.0,1.0,1.0


In [9]:
# join the two categorical dfs into one

marketing_categorical = pd.concat([marketing_ordinal, marketing_nominal], axis=1)

In [10]:
# join categorical with numerical

mark_df = pd.concat([marketing_categorical, marketing_numerical], axis=1)

# customer and effective_to_date columns not included

### X-y split
(y is the target variable, in this case, "total claim amount")

In [11]:
mark_df = mark_df.dropna()
mark_df = mark_df.drop_duplicates()

In [12]:
import matplotlib.pyplot as plt 

In [13]:
X = mark_df.drop('total_claim_amount', axis=1)
y = mark_df[['total_claim_amount']]

### Train-test split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

### Standardize the data

In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

In [19]:
# apply PowerTransformer to X_train
pt = PowerTransformer()
pt.fit(X_train)
X_train_trans = pt.transform(X_train)
X_train_trans = pd.DataFrame(X_train_trans)
X_train_trans.columns = X.columns

In [20]:
# transform the test data with the same PowerTransformer
X_test_trans = pt.transform(X_test)
X_test_trans = pd.DataFrame(X_test_trans)
X_test_trans.columns = X.columns

,response,coverage,education,vehicle_size,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,employmentstatus_Disabled,...,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,month
0,-0.372409,-0.785999,-0.102471,0.145520,-0.467560,1.315308,-0.319839,-0.616152,-0.307391,-0.217833,...,-0.229947,2.038162,-0.053983,-1.634083,-0.506353,-1.511360,-0.402996,-0.574243,1.674285,-0.939687
1,-0.372409,-0.785999,-0.102471,0.145520,-0.467560,1.315308,-0.319839,-0.616152,-0.307391,-0.217833,...,-0.229947,2.038162,-1.338619,0.194961,-0.703169,-0.747314,0.059452,-0.574243,-1.152927,1.064184
2,-0.372409,-0.785999,-0.102471,-1.711888,-0.467560,-0.760278,3.126572,-0.616152,-0.307391,-0.217833,...,-0.229947,-0.490638,-0.092380,0.990945,-0.845993,0.664108,1.214726,-0.574243,1.465122,1.064184
3,-0.372409,-0.785999,0.744079,0.145520,-0.467560,-0.760278,-0.319839,1.622977,-0.307391,-0.217833,...,-0.229947,-0.490638,0.498519,0.676750,0.710053,0.141409,0.949768,1.253877,1.465122,1.064184
4,-0.372409,1.550475,-0.102471,0.145520,-0.467560,1.315308,-0.319839,-0.616152,-0.307391,-0.217833,...,-0.229947,-0.490638,1.380008,-1.634083,1.694042,-0.172852,1.156563,1.782126,-0.067457,-0.939687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,2.685221,-0.785999,-1.341749,2.107796,2.138765,-0.760278,-0.319839,-0.616152,-0.307391,-0.217833,...,-0.229947,2.038162,1.605078,1.051031,-0.703169,0.314205,-1.184730,-0.574243,-0.067457,-0.939687
2572,-0.372409,-0.785999,-1.341749,0.145520,-0.467560,1.315308,-0.319839,-0.616152,-0.307391,-0.217833,...,-0.229947,-0.490638,-0.002548,0.910534,-0.569780,0.664108,-1.772470,-0.574243,1.674285,-0.939687
2573,-0.372409,-0.785999,0.744079,0.145520,-0.467560,1.315308,-0.319839,-0.616152,-0.307391,4.590674,...,-0.229947,-0.490638,-0.080113,0.088275,-0.506353,-1.979195,1.639346,1.782126,0.513327,-0.939687
2574,-0.372409,1.550475,-1.341749,0.145520,-0.467560,-0.760278,3.126572,-0.616152,-0.307391,-0.217833,...,-0.229947,-0.490638,0.705327,1.079664,1.044939,1.580049,1.127333,-0.574243,1.320685,-0.939687


In [21]:
# transform target variables y_train and y_test as well
scaler = StandardScaler()
scaler.fit(y_train)

y_train_trans = scaler.transform(y_train)
y_train_trans = pd.DataFrame(y_train_trans)

y_test_trans = scaler.transform(y_test)
y_test_trans = pd.DataFrame(y_test_trans)

#scaler.inverse_transform(y_test)
#scaler.inverse_transform(y_pred)

### Linear Model

### Apply linear regression
i.e. train the model

In [23]:
from sklearn.linear_model import LinearRegression

In [24]:
# find the line of best fit, i.e. the intercept and slope of our regression line
lm = LinearRegression()
linear_model = lm.fit(X_train_trans,y_train_trans) 

In [25]:
linear_model.coef_

array([[-2.61505293e-02,  6.58027327e-02, -1.36665541e-02,
        -5.68733250e-03, -1.47275189e+11, -1.84934214e+11,
        -1.11375830e+11, -1.71426906e+11, -1.07804739e+11,
        -2.37845670e+11, -5.54204467e+11, -2.39778101e+11,
        -1.93960042e+11, -4.98974287e+11, -4.87556537e+11,
        -4.87556537e+11, -4.22314877e+12, -5.10451625e+12,
        -3.97187091e+12,  1.11616609e+12,  1.55297662e+12,
         1.40225236e+12,  7.44601384e+12,  8.46322375e+12,
         5.06692917e+12, -1.50989607e+11, -1.94432157e+11,
        -2.52035674e+11, -7.03934724e+11, -8.64760148e+11,
        -9.91029467e+11, -6.99371355e+11, -1.07161687e+12,
        -1.00958730e+12, -2.71219788e+12, -2.56980208e+12,
        -2.03295951e+12, -1.74188764e+12, -1.44635430e+12,
        -1.33258749e+12, -1.17652315e+12, -1.05929978e+12,
         1.50769128e+12,  4.01418679e+11,  4.13964685e+11,
         1.18693215e+12,  6.59568417e+11,  1.19425037e+12,
         1.68457031e-02, -3.51562500e-02,  3.02734375e-0

In [26]:
linear_model.intercept_

array([-0.00202623])

### Model Interpretation

In [29]:
# look at a random customer
random_customer = X_test_trans.sample()
scaler.inverse_transform(linear_model.predict(random_customer))

,response,coverage,education,vehicle_size,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,employmentstatus_Disabled,...,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,month
1224,2.685221,-0.785999,-0.102471,0.14552,-0.46756,-0.760278,-0.319839,-0.616152,3.253184,-0.217833,...,-0.229947,-0.490638,-1.481403,0.441512,-0.845993,-0.069932,-1.18473,-0.574243,-1.152927,1.064184


In [37]:
# Create y_pred
y_pred = linear_model.predict(X_test_trans) 
y_pred = pd.DataFrame(scaler.inverse_transform(y_pred))
y_pred.columns = ['y_pred']
y_pred


,y_pred
0,470.784838
1,388.434606
2,219.029215
3,415.910651
4,854.146204
...,...
2571,370.049878
2572,29.483693
2573,396.000918
2574,414.313304


In [34]:
y_test_trans = y_test_trans.reset_index(drop=True)
y_test_inversed = pd.DataFrame(scaler.inverse_transform(y_test_trans))
y_test_inversed.columns = ['y_test']
y_test_inversed

,y_test
0,432.408100
1,511.200000
2,325.395532
3,480.578584
4,1452.213743
...,...
2571,379.029102
2572,17.375878
2573,355.200000
2574,553.503610


In [39]:
# calculate residuals of test data via y_test and y_pred 
residuals_test = pd.concat([y_test_inversed,y_pred],axis=1)
residuals_test['residual'] = residuals_test['y_test']-residuals_test['y_pred']
residuals_test 

,y_test,y_pred
0,432.408100,470.784838
1,511.200000,388.434606
2,325.395532,219.029215
3,480.578584,415.910651
4,1452.213743,854.146204
...,...,...
2571,379.029102,370.049878
2572,17.375878,29.483693
2573,355.200000,396.000918
2574,553.503610,414.313304


In [81]:
# calculate residuals of train data via y_train and y_pred_train
y_train_trans.reset_index(drop=True,inplace=True)
y_train_inversed = pd.DataFrame(scaler.inverse_transform(y_train_trans))
y_train_inversed.columns = y_train.columns

y_pred_train = linear_model.predict(X_train_trans) 
y_pred_inversed = pd.DataFrame(scaler.inverse_transform(y_pred_train))
y_pred_inversed.columns = ['y_pred_train']

residuals_train = pd.concat([y_train_inversed,y_pred_inversed],axis=1).reset_index(drop=True)
residuals_train.rename(columns={'total_claim_amount': 'y_train'},inplace=True)
residuals_train['residual'] = residuals_train['y_train']-residuals_train['y_pred_train']
residuals_train


,y_train,y_pred_train,residual
0,251.429988,299.792076,-48.362088
1,200.177163,177.581167,22.595996
2,285.825083,375.472084,-89.647001
3,297.600000,382.575186,-84.975186
4,25.478179,-36.323070,61.801249
...,...,...,...
7721,0.769185,245.555151,-244.785966
7722,36.980596,207.506909,-170.526313
7723,347.075948,498.544847,-151.468899
7724,236.842746,209.469833,27.372913


In [47]:
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae

In [59]:
# calculating error metrics for test data

lm_me_test = round(residuals_test.residual.mean(),2)
print(f'mean error of test: {lm_me_test}')

lm_mse_test = round(mse(y_test_inversed,y_pred),2) # mean squared error
print(f'mean squared error of test: {lm_mse_test}')

lm_mae_test = round(mae(y_test_inversed,y_pred),2) # mean absolute error
print(f'mean absolut error of test: {lm_mae_test}')

lm_rmse_test = round(mse(y_test_inversed,y_pred,squared=False),2) # root mean squared error
print(f'root mean squared error of test: {lm_rmse_test}')

mean error of test: 2.54
mean squared error of test: 21187.62
mean absolut error of test: 99.27
root mean squared error of test: 145.56


In [82]:
# calculating error metrics for train data

lm_me_test2 = round(residuals_train.residual.mean(),2)
print(f'mean error of test: {lm_me_test2}')

lm_mse_test2 = round(mse(y_train_inversed,y_pred_inversed),2) # mean squared error
print(f'mean squared error of test: {lm_mse_test2}')

lm_mae_test2 = round(mae(y_train_inversed,y_pred_inversed),2) # mean absolute error
print(f'mean absolut error of test: {lm_mae_test2}')

lm_rmse_test2 = round(mse(y_train_inversed,y_pred_inversed,squared=False),2) # root mean squared error
print(f'root mean squared error of test: {lm_rmse_test2}')

mean error of test: -0.16
mean squared error of test: 22724.54
mean absolut error of test: 101.85
root mean squared error of test: 150.75


In [49]:
from sklearn.metrics import r2_score

In [83]:
# calculating R2 score for test data

lm_r2_test = round(r2_score(y_test_inversed,y_pred),2)
lm_r2_test

0.76

In [84]:
# calculating R2 score for train data

lm_r2_train = round(r2_score(y_train_inversed,y_pred_inversed),2)
lm_r2_train

# compared to 0.43 and 0.45 from only the original numerical columns from Tuesday
# R2 has increased with additional input variables after encoding categorical columns

0.73

## Week 7 Lab Work

### 1. Fit the models LinearRegression, Lasso and Ridge and compare the model performances.

<b>Linear Regression</b><br>
    Refer back to Model 1

In [79]:
# model performance metrics from Model 1 using PowerTransformer

print(f"{linear_model.__class__.__name__}: Train -> {linear_model.score(X_train_trans, y_train_trans)}, Test -> {linear_model.score(X_test_trans,y_test_trans)}")

LinearRegression: Train -> 0.7328467291006477, Test -> 0.7637963081379076


<b>Lasso Regression

In [86]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
import numpy as np

In [101]:
# find best alpha for lasso model

# generate 20 values between 10ˆ-4 and 10ˆ2
alphas_lasso = np.logspace(-4, 2, 20)

lasso_best_a = None
lasso_best_score = 0

# evaluate over all alpha values in range
for a in alphas_lasso:
    lasso_model = Lasso(alpha = a)
    lasso_model.fit(X_train, y_train)
    lasso_score = lasso_model.score(X_train, y_train)
    
    # keep track of best score
    if lasso_score > lasso_best_score:
        lasso_best_score = lasso_score
        lasso_best_a = a

print(f"The best alpha value for the lasso model is: {lasso_best_a}\n with a score of: {lasso_best_score}")


/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.877e+07, tolerance: 6.572e+04
  model = cd_fast.enet_coordinate_descent(
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.877e+07, tolerance: 6.572e+04
  model = cd_fast.enet_coordinate_descent(
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

The best alpha value for the lasso model is: 0.0001
 with a score of: 0.7627246203269168


In [105]:
# run the lasso regression model using lasso_best_a

lasso_model=Lasso(alpha=lasso_best_a)
lasso_model.fit(X_train, y_train)
print(f"{lasso_model.__class__.__name__}: Train -> {lasso_model.score(X_train, y_train)}, Test -> {lasso_model.score(X_test, y_test)}")

Lasso: Train -> 0.7627246203269168, Test -> 0.7952982007466616


/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.877e+07, tolerance: 6.572e+04
  model = cd_fast.enet_coordinate_descent(


<b>Ridge Regression

In [106]:
# find best alpha for ridge model

# generate 20 values between 10ˆ-4 and 10ˆ2
alphas_ridge = np.logspace(-4, 2, 20)

ridge_best_a = None
ridge_best_score = 0

# evaluate over all alpha values in range
for a in alphas_ridge:
    ridge_model = Ridge(alpha = a)
    ridge_model.fit(X_train, y_train)
    ridge_score = ridge_model.score(X_train, y_train)
    
    # keep track of best score
    if ridge_score > ridge_best_score:
        ridge_best_score = ridge_score
        ridge_best_a = a

print(f"The best alpha value for the ridge model is: {ridge_best_a}\n with a score of: {ridge_best_score}")

## QUESTION: Is it normal that the ridge_best_a is the same as lasso_best_a??

The best alpha value for the ridge model is: 0.0001
 with a score of: 0.7627246205419619


/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.3914e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.87901e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.95706e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [107]:
# run the ridge regression model using ridge_best_a
ridge_model=Ridge(alpha=ridge_best_a) 
ridge_model.fit(X_train, y_train)
print(f"{ridge_model.__class__.__name__}: Train -> {ridge_model.score(X_train, y_train)}, Test -> {ridge_model.score(X_test, y_test)}")

Ridge: Train -> 0.7627246205419619, Test -> 0.7952973118054274


/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.3914e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


### 2. Define a function that takes a list of models and trains (and tests) them so we can try a lot of them without repeating code.

In [115]:
def compare_models(X_train, X_test, y_train, y_test):

    """ range of alphas to test lasso and ridge models """
    alphas = np.logspace(-4, 2, 20)

    """ find ideal alpha for lasso model """
    lasso_best_a = None
    lasso_best_score = 0

    for a in alphas:
        lasso_model = Lasso(alpha = a)
        lasso_model.fit(X_train, y_train)
        lasso_score = lasso_model.score(X_train, y_train)

        # keep track of best score
        if lasso_score > lasso_best_score:
            lasso_best_score = lasso_score
            lasso_best_a = a

    """ find ideal alpha for ridge model """
    ridge_best_a = None
    ridge_best_score = 0

    for a in alphas:
        ridge_model = Ridge(alpha = a)
        ridge_model.fit(X_train, y_train)
        ridge_score = ridge_model.score(X_train, y_train)

        # keep track of best score
        if ridge_score > ridge_best_score:
            ridge_best_score = ridge_score
            ridge_best_a = a

    """compare linear, lasso and ridge models"""
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    linear_score = linear_model.score(X_train, y_train)
    print(f"{linear_model} gives us an r-squared score of {linear_score}.\n")

    lasso_model = Lasso(alpha=lasso_best_a)
    lasso_model.fit(X_train, y_train)
    lasso_score = lasso_model.score(X_train, y_train)
    print(f"{lasso_model} gives us an r-squared score of {lasso_score}.\n")

    ridge_model = Ridge(alpha=ridge_best_a)
    ridge_model.fit(X_train, y_train)
    ridge_score = ridge_model.score(X_train, y_train)
    print(f"{ridge_model} gives us an r-squared score of {ridge_score}.\n")

    if linear_score > lasso_score:
        if linear_score > ridge_score:
            best_model = linear_model
            print(f"The linear model provides the highest r-squared score of {linear_score}.\n")
        elif ridge_score > lasso_score:
            best_model = ridge_model
            print(f"The ridge model provides the highest r-squared score of {ridge_score}.\n")
    else:
        best_model = lasso_model
        print(f"The lasso model provides the highest r-squared score of {lasso_score}.\n")
    
    return best_model

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 56)

# apply PowerTransformer to X_train and X_test
pt = PowerTransformer()
pt.fit(X_train)

X_train = pt.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

X_test = pt.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

# apply StandardScaler y_train and y_test
scaler = StandardScaler()
scaler.fit(y_train)

y_train = scaler.transform(y_train)
y_train = pd.DataFrame(y_train)

y_test = scaler.transform(y_test)
y_test = pd.DataFrame(y_test)

In [118]:
best_model = compare_models(X_train, X_test, y_train, y_test)
best_model

/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.098e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.133e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(


LinearRegression() gives us an r-squared score of 0.7368764175571072.

Lasso(alpha=0.0001) gives us an r-squared score of 0.7400634429470287.

Ridge(alpha=0.0001) gives us an r-squared score of 0.740063892900773.

The lasso model provides the highest r-squared score of 0.7400634429470287.



/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.098e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.0001)

Try again with a StandardScaler...

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 56)

# apply StandardScaler to X_train and X_test
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

# apply StandardScaler y_train and y_test
scaler = StandardScaler()
scaler.fit(y_train)

y_train = scaler.transform(y_train)
y_train = pd.DataFrame(y_train)

y_test = scaler.transform(y_test)
y_test = pd.DataFrame(y_test)

In [137]:
best_model = compare_models(X_train, X_test, y_train, y_test)

/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.606e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(
/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.636e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(


LinearRegression() gives us an r-squared score of 0.7683069224755956.

Lasso(alpha=0.0001) gives us an r-squared score of 0.7713084691645632.

Ridge(alpha=0.0001) gives us an r-squared score of 0.7713093502417152.

The lasso model provides the highest r-squared score of 0.7713084691645632.



/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.606e+01, tolerance: 7.726e-01
  model = cd_fast.enet_coordinate_descent(


Try again with MinMaxScaler...

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 56)

# apply MinMaxScaler to X_train and X_test
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

# apply StandardScaler y_train and y_test
scaler = StandardScaler()
scaler.fit(y_train)

y_train = scaler.transform(y_train)
y_train = pd.DataFrame(y_train)

y_test = scaler.transform(y_test)
y_test = pd.DataFrame(y_test)

In [135]:
best_model = compare_models(X_train, X_test, y_train, y_test)

LinearRegression() gives us an r-squared score of 0.7709647967120321.

Lasso(alpha=0.0001) gives us an r-squared score of 0.7712886192479993.

Ridge(alpha=0.0001) gives us an r-squared score of 0.7713093502400163.

The lasso model provides the highest r-squared score of 0.7712886192479993.



<b>Lasso Model</b> seems to consistently provide better results regardless of the scaler used on the train data

### 3. Use feature selection techniques (P-Value, RFE) to select a subset of features to train the model with.

<b>P-Value selection technique

In [ ]:
# add a constant term(intercept) to the feature matrix X
# fit a linear regression model using ordinary least squares
# extract p_values for each coefficient, excluding constant term
# select features whos p-values are less than the specified signficance level
# return list of selected featuers

In [123]:
import statsmodels.api as sm

In [149]:
# Add a constant to the X_train for the intercept term
X_train = sm.add_constant(X_train)

# Fit the OLS model
ols_model = sm.OLS(y_train, X_train).fit()
display(ols_model.summary())
print(f"The r-squared score of this OLS model without feature selection is {ols_model.rsquared}")

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      0   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     586.9
Date:                Tue, 20 Feb 2024   Prob (F-statistic):               0.00
Time:                        12:45:09   Log-Likelihood:                -5272.9
No. Observations:                7726   AIC:                         1.064e+04
Df Residuals:                    7681   BIC:                         1.095e+04
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0039      0.006      0.681      0.496      -0.007       0.015
response                          -0.0283      0.008     -3.649      0.000      -0.044      -0.013
coverage                          -0.0181      0.010     -1.842      0.066      -0.037       0.001
education                         -0.0211      0.006     -3.693      0.000      -0.032      -0.010
vehicle_size                      -0.0013      0.006     -0.226      0.821      -0.012       0.010
state_Arizona                   2.629e+11   1.69e+11      1.558      0.119   -6.79e+10    5.94e+11
state_California                3.315e+11   2.13e+11      1.558      0.119   -8.56e+10    7.48e+11
state_Nevada                    1.992e+11   1.28e+11      1.558      0.119   -5.14e+10     4.5e+11
state_Oregon                    3.052e+11   1.96e+11      1.558      0.119   -7.88e+10    6.89e+11
state_Washington                1.933e+11   1.24e+11      1.558      0.119   -4.99e+10    4.37e+11
employmentstatus_Disabled      -4.288e+11   5.76e+11     -0.744      0.457   -1.56e+12       7e+11
employmentstatus_Employed      -9.999e+11   1.34e+12     -0.744      0.457   -3.63e+12    1.63e+12
employmentstatus_Medical Leave -4.346e+11   5.84e+11     -0.744      0.457   -1.58e+12     7.1e+11
employmentstatus_Retired       -3.494e+11   4.69e+11     -0.744      0.457   -1.27e+12    5.71e+11
employmentstatus_Unemployed    -8.994e+11   1.21e+12     -0.744      0.457   -3.27e+12    1.47e+12
gender_F                         1.52e+11   5.84e+11      0.260      0.795   -9.93e+11     1.3e+12
gender_M                         1.52e+11   5.84e+11      0.260      0.795   -9.93e+11     1.3e+12
location_code_Rural             1.752e+12    8.3e+11      2.113      0.035    1.26e+11    3.38e+12
location_code_Suburban          2.141e+12   1.01e+12      2.113      0.035    1.54e+11    4.13e+12
location_code_Urban             1.687e+12   7.99e+11      2.113      0.035    1.22e+11    3.25e+12
marital_status_Divorced         3.449e+11   6.95e+11      0.496      0.620   -1.02e+12    1.71e+12
marital_status_Married          4.815e+11    9.7e+11      0.496      0.620   -1.42e+12    2.38e+12
marital_status_Single           4.351e+11   8.77e+11      0.496      0.620   -1.28e+12    2.15e+12
policy_type_Corporate Auto      1.475e+12   9.87e+11      1.495      0.135   -4.59e+11    3.41e+12
policy_type_Personal Auto       1.127e+12   1.15e+12      0.978      0.328   -1.13e+12    3.38e+12
policy_type_Special Auto        4.977e+11   2.34e+11      2.123      0.034    3.81e+10    9.57e+11
policy_Corporate L1            -7.933e+10   2.96e+11     -0.268      0.789   -6.59e+11       5e+11
policy_Corporate L2            -1.004e+11   3.74e+11     -0.268      0.789   -8.34e+11    6.33e+11
policy_Corpora

The r-squared score of this OLS model without feature selection is 0.7707388827333224


In [155]:
p_values = ols_model.pvalues
p_values

const                              4.958399e-01
response                           2.647001e-04
coverage                           6.558148e-02
education                          2.229007e-04
vehicle_size                       8.208910e-01
state_Arizona                      1.192522e-01
state_California                   1.192522e-01
state_Nevada                       1.192522e-01
state_Oregon                       1.192522e-01
state_Washington                   1.192522e-01
employmentstatus_Disabled          4.566368e-01
employmentstatus_Employed          4.566368e-01
employmentstatus_Medical Leave     4.566368e-01
employmentstatus_Retired           4.566368e-01
employmentstatus_Unemployed        4.566368e-01
gender_F                           7.946551e-01
gender_M                           7.946551e-01
location_code_Rural                3.467524e-02
location_code_Suburban             3.467524e-02
location_code_Urban                3.467524e-02
marital_status_Divorced            6.197

In [142]:
# Filter out features with p-values above 0.05
selected_features = p_values[p_values < 0.05].index

# Select only the features with p-values below the threshold
X_train_selected = X_train[selected_features]

selected_features

# can rerun OLS with X_train_selected

Index(['response', 'education', 'location_code_Rural',
       'location_code_Suburban', 'location_code_Urban',
       'policy_type_Special Auto', 'sales_channel_Agent',
       'sales_channel_Branch', 'sales_channel_Call Center',
       'sales_channel_Web', 'vehicle_class_Four-Door Car',
       'vehicle_class_Luxury Car', 'vehicle_class_Luxury SUV',
       'vehicle_class_SUV', 'vehicle_class_Sports Car',
       'vehicle_class_Two-Door Car', 'income', 'monthly_premium_auto'],
      dtype='object')

In [150]:
ols_model2 = sm.OLS(y_train, X_train_selected).fit()
display(ols_model2.summary())
print(f"The r-squared score of this OLS model with feature selection is {ols_model2.rsquared}")

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      0   R-squared (uncentered):                   0.749
Model:                            OLS   Adj. R-squared (uncentered):              0.749
Method:                 Least Squares   F-statistic:                              1357.
Date:                Tue, 20 Feb 2024   Prob (F-statistic):                        0.00
Time:                        12:46:30   Log-Likelihood:                         -5615.4
No. Observations:                7726   AIC:                                  1.126e+04
Df Residuals:                    7709   BIC:                                  1.138e+04
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
response                       -0.0376      0.006     -6.379      0.000      -0.049      -0.026
education                      -0.0292      0.006     -4.985      0.000      -0.041      -0.018
location_code_Rural         -5.959e+10   8.26e+10     -0.722      0.471   -2.21e+11    1.02e+11
location_code_Suburban      -7.281e+10   1.01e+11     -0.722      0.471   -2.71e+11    1.25e+11
location_code_Urban         -5.736e+10   7.95e+10     -0.722      0.471   -2.13e+11    9.85e+10
policy_type_Special Auto        0.0106      0.006      1.864      0.062      -0.001       0.022
sales_channel_Agent          1.026e+11   2.99e+11      0.343      0.731   -4.83e+11    6.89e+11
sales_channel_Branch         9.561e+10   2.78e+11      0.343      0.731    -4.5e+11    6.41e+11
sales_channel_Call Center    8.362e+10   2.44e+11      0.343      0.731   -3.94e+11    5.61e+11
sales_channel_Web            7.508e+10   2.19e+11      0.343      0.731   -3.54e+11    5.04e+11
vehicle_class_Four-Door Car  1.733e+11   7.51e+11      0.231      0.817    -1.3e+12    1.64e+12
vehicle_class_Luxury Car      4.53e+10   1.96e+11      0.231      0.817   -3.39e+11     4.3e+11
vehicle_class_Luxury SUV     4.709e+10   2.04e+11      0.231      0.817   -3.53e+11    4.47e+11
vehicle_class_SUV            1.355e+11   5.87e+11      0.231      0.817   -1.01e+12    1.29e+12
vehicle_class_Sports Car     7.615e+10    3.3e+11      0.231      0.817    -5.7e+11    7.23e+11
vehicle_class_Two-Door Car   1.391e+11   6.02e+11      0.231      0.817   -1.04e+12    1.32e+12
income                         -0.1327      0.006    -20.706      0.000      -0.145      -0.120
monthly_premium_auto            0.5931      0.011     55.133      0.000       0.572       0.614
==============================================================================
Omnibus:                     3736.588   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48100.218
Skew:                           1.992   Prob(JB):                         0.00
Kurtosis:                      14.556   Cond. No.                     3.18e+14
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 1.72e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

The r-squared score of this OLS model with feature selection is 0.7494827337492969


We get a lower r-squared with feature selection...

<b>Recursive feature elimination technique

In [151]:
from sklearn.feature_selection import RFE 

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=41)

# scale our train data
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

# apply linear model for RFE with 10 features
rfe_lm = LinearRegression()

selector = RFE(rfe_lm, n_features_to_select=48, step = 2, verbose = 1)
selector.fit(X_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

# run linear regression using kept_features
rfe_lm.fit(X_train, y_train)

print(f"Final model with selected features has an r-squared score of {rfe_lm.score(X_train, y_train)}: ")
display(X_train)

Fitting estimator with 56 features.
Fitting estimator with 54 features.
Fitting estimator with 52 features.
Fitting estimator with 50 features.
Final model with selected features has an r-squared score of 0.7709128553687966: 


,response,coverage,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,employmentstatus_Disabled,employmentstatus_Employed,employmentstatus_Medical Leave,...,sales_channel_Call Center,sales_channel_Web,vehicle_class_Four-Door Car,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,income,monthly_premium_auto
0,-0.372256,-0.729356,-0.470181,-0.772117,-0.322416,1.651446,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,0.942552,-0.128076,-0.138452,-0.480349,-0.233475,-0.494267,-0.043352,-0.739399
1,-0.372256,0.798122,-0.470181,1.295140,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,0.942552,-0.128076,-0.138452,-0.480349,-0.233475,-0.494267,1.232015,-0.092005
2,-0.372256,-0.729356,2.126842,-0.772117,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,0.942552,-0.128076,-0.138452,-0.480349,-0.233475,-0.494267,0.881764,-0.768826
3,-0.372256,0.798122,-0.470181,-0.772117,-0.322416,1.651446,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,2.424658,-1.060949,-0.128076,-0.138452,2.081821,-0.233475,-0.494267,1.924980,0.849658
4,-0.372256,-0.729356,-0.470181,-0.772117,-0.322416,1.651446,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,0.942552,-0.128076,-0.138452,-0.480349,-0.233475,-0.494267,0.364468,-0.768826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8236,-0.372256,-0.729356,-0.470181,1.295140,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,2.031010,-0.412429,0.942552,-0.128076,-0.138452,-0.480349,-0.233475,-0.494267,1.293698,-0.739399
8237,-0.372256,-0.729356,-0.470181,1.295140,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,-1.060949,-0.128076,-0.138452,-0.480349,-0.233475,2.023197,-0.413385,-0.621691
8238,-0.372256,0.798122,-0.470181,1.295140,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,-1.060949,-0.128076,-0.138452,-0.480349,-0.233475,2.023197,1.833738,-0.445129
8239,-0.372256,-0.729356,-0.470181,1.295140,-0.322416,-0.605530,-0.300852,-0.215275,0.775725,-0.216813,...,-0.492366,-0.412429,-1.060949,-0.128076,-0.138452,-0.480349,4.283115,-0.494267,1.569594,0.702523
